# Face Detection

In [42]:
#import necessary modules

import numpy as np
import pandas as pd
import cv2
import os

## Load images from dir


In [43]:
#images path
img_dir = '/workspaces/faceid/train_images'
# Create a mapping from folder names to integer labels
label_map = {'wmaathai': 1, 'others': 0}

## Train FaceRecognizer Model

In [44]:
# initialize empty lists for training images and image-labels
faces = []
ids = []

In [ ]:
# train FaceRecognizer model
for face_id in os.listdir(img_dir):
    face_path = os.path.join(img_dir, face_id)
    
    if os.path.isdir(face_path): # Check if it's a directory (person's folder)
        # Check if the folder name is in the label_map
        if face_id in label_map:
            label = label_map[face_id]
        else:
            continue  # Skip folders not in the label map

        # process folder images
        for img_label in os.listdir(face_path):
            img_path = os.path.join(face_path, img_label)
            
            img = cv2.imread(img_path)

            #skip invalid images
            if img is None:
                print(f"Failed to load image {img_path}")
                continue

            # convert image to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # load Haar CascadeClasifier and detect faces
            face_cascade = cv2.CascadeClassifier('/workspaces/faceid/haarcascades/haarcascade_frontalface_default.xml')
            faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5) #minSize=(40, 40)

            print(f"Detected {len(faces_detected)} face(s) in {img_path}")

            # Process each face detected
            for (x, y, w, h) in faces_detected:
                face = gray_img[y:y+h, x:x+w]  # Extract face region
                faces.append(face)  # Add the face to the list
                ids.append(int(label))  # Add the corresponding label (ID)


Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/8.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/22.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/11.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/13.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/27.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/9.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/16.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/24.jpg


Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/21.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/23.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/5.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/3.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/14.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/26.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/1.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/4.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/15.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/20.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/7.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/2.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/12.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/25.jpg
Detected 0 face(s) in /wor

## Face Detection

In [46]:
# processing images
"""
(x, y) is the top-left corner of the rectangle
(w, h) is the width and height of the rectangle
"""
for (x, y, w, h) in faces_detected: # face bounding box
    face = gray_img[y:y+h, x:x+w] # extract face from gray_img
    faces.append(face) # add extracted face to list
    ids.append(int(face_id))  # Label with the id (folder name)
    #label, confidence = recognizer.predict(face_region)

In [47]:
"""
print(faces)
print(ids)
# Check if faces and ids are empty
if len(faces) == 0 or len(ids) == 0:
    print("No faces detected or empty data.")
else:
    print(f"Training data: {len(faces)} faces with {len(ids)} labels.")
"""

'\nprint(faces)\nprint(ids)\n# Check if faces and ids are empty\nif len(faces) == 0 or len(ids) == 0:\n    print("No faces detected or empty data.")\nelse:\n    print(f"Training data: {len(faces)} faces with {len(ids)} labels.")\n'

In [48]:
# Train the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(ids))


In [49]:
# Save the trained model
recognizer.save('trained_model.yml')
print("Model trained and saved successfully.")

Model trained and saved successfully.


In [50]:
# Recognize the face using the pre-trained face recognizer
                label, confidence = recognizer.predict(face_region)

IndentationError: unexpected indent (1165184431.py, line 2)